In [101]:
# import libraries and env variables
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
APP_TOKEN = os.environ.get("X-App-Token")


In [178]:
# initialize parameters
start_date = '2023-11-06T00:00:00.000'
end_date = '2023-11-19T23:59:59.999'
soql_date = f"where=date_of_occurrence between '{start_date}' and '{end_date}'" #9937 records
limit = 1000

response_data = []

i = 0
while True:
    offset = i * limit # offset = 0, 1000, 2000, etc.
    response = requests.get(f"https://data.cityofchicago.org/resource/x2n5-8w5q.json?"
                            f"$$app_token={APP_TOKEN}"
                            f"&$order=:id"  
                            f"&${soql_date}"
                            f"&$limit={limit}"
                            f"&$offset={offset}"
                            f"&$select=:*,*") #include metadata field info

    if not response.status_code==200:
        raise Exception

    if response.json() == []:
        break

    response_data.extend(response.json())
    i += 1

crime_df = pd.json_normalize(data=response_data)

In [173]:
response.headers

{'Server': 'nginx', 'Date': 'Sun, 14 Jan 2024 15:18:30 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'ETag': '"YWxwaGEuMTAzMTZfOF8xOTcwMmN3ZVBPekRpRkt6dFNRTWcyRzcyekc4Xzhtaw---gzioBkxGR4JIFuBj0YDtMZMeX2ENhG--gzip--gzip"', 'X-SODA2-Fields': '[":id",":created_at",":updated_at",":version",":@computed_region_awaf_s7ux",":@computed_region_6mkv_f3dw",":@computed_region_vrxf_vc4k",":@computed_region_bdys_3d7i",":@computed_region_43wa_7qmu",":@computed_region_rpca_8um6","case_","date_of_occurrence","block","_iucr","_primary_decsription","_secondary_description","_location_description","arrest","domestic","beat","ward","fbi_cd","x_coordinate","y_coordinate","latitude","longitude","location"]', 'X-SODA2-Types': '["row_identifier","fixed_timestamp","fixed_timestamp","row_version","number","number","number","number","number","number","text","floating_timestamp","text","text","text","text","tex

In [179]:
response.json()

[]

In [180]:
print(f"Distinct entries: {len(crime_df.groupby([':id']))}")

Distinct entries: 9937


In [181]:
crime_df = crime_df.drop(columns=[':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw',
       ':@computed_region_vrxf_vc4k', ':@computed_region_bdys_3d7i',
       ':@computed_region_43wa_7qmu', ':@computed_region_rpca_8um6'])

In [165]:
crime_df.columns

Index([':id', ':created_at', ':updated_at', ':version', 'case_',
       'date_of_occurrence', 'block', '_iucr', '_primary_decsription',
       '_secondary_description', '_location_description', 'arrest', 'domestic',
       'beat', 'ward', 'fbi_cd', 'x_coordinate', 'y_coordinate', 'latitude',
       'longitude', 'location.latitude', 'location.longitude',
       'location.human_address'],
      dtype='object')

In [182]:
crime_df

,:id,:created_at,:updated_at,:version,case_,date_of_occurrence,block,_iucr,_primary_decsription,_secondary_description,...,beat,ward,fbi_cd,x_coordinate,y_coordinate,latitude,longitude,location.latitude,location.longitude,location.human_address
0,row-n56f.jhj4-rhq4,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-fhmb.32vx_5wns,JG495930,2023-11-06T00:00:00.000,023XX W LELAND AVE,0820,THEFT,$500 AND UNDER,...,1911,47,06,1159788,1931121,41.96671217,-87.687866208,41.96671217,-87.687866208,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
1,row-p4xb~y53j-fuek,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-5cs2.q96g.nma6,JG494364,2023-11-06T00:00:00.000,105XX S WALDEN PKWY,0560,ASSAULT,SIMPLE,...,2212,19,08A,1165481,1834708,41.702025415,-87.669672767,41.702025415,-87.669672767,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2,row-bevv.9vv5~9xk5,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-qf9x~yuq5.p9r6,JG495115,2023-11-06T00:00:00.000,074XX W BELMONT AVE,0930,MOTOR VEHICLE THEFT,THEFT / RECOVERY - AUTOMOBILE,...,1631,29,07,1125766,1920416,41.937971562,-87.813201099,41.937971562,-87.813201099,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
3,row-u3gy~wdhi.hd3k,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-jk7w~dau9-eci4,JG494379,2023-11-06T00:00:00.000,069XX S HARPER AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,...,332,5,08A,1187589,1859066,41.768370253,-87.587947949,41.768370253,-87.587947949,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
4,row-m3c2~ztbv~kdrh,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-ma52_veqi_ib8b,JG493406,2023-11-06T00:00:00.000,098XX S HOXIE AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,...,431,7,07,1195083,1840238,41.71652318,-87.561098665,41.71652318,-87.561098665,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,row-yswx.d7ji~6bbm,2024-01-14T11:02:14.225Z,2024-01-14T11:02:21.674Z,rv-zzqz_pcgz~nmvr,JH112431,2023-11-10T00:00:00.000,006XX W ARLINGTON PL,0810,THEFT,OVER $500,...,1935,43,06,1171556,1916635,41.926710647,-87.645024988,41.926710647,-87.645024988,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
9933,row-heuz.tfm6~msyh,2024-01-14T11:02:14.225Z,2024-01-14T11:02:21.674Z,rv-czr3~pznu.anmy,JH112354,2023-11-06T12:00:00.000,019XX S MAY ST,0460,BATTERY,SIMPLE,...,1235,25,08B,1169097,1890994,41.856403828,-87.654805728,41.856403828,-87.654805728,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
9934,row-nr4y~hg29_y6kj,2024-01-14T11:02:14.225Z,2024-01-14T11:02:21.674Z,rv-85ru~zme6~7t2e,JH113345,2023-11-16T07:40:00.000,026XX N NARRAGANSETT AVE,0860,THEFT,RETAIL THEFT,...,2512,36,06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9935,row-gue5~wdzu-kf2k,2024-01-14T11:02:14.225Z,2024-01-14T11:02:21.674Z,rv-r2pf.7k6u~yr7e,JH112145,2023-11-15T12:00:00.000,024XX W MADISON ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,...,1125,27,11,1160180,1899944,41.881152194,-87.687288534,41.881152194,-87.687288534,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."


In [105]:
# create a dataframe for specific dates with holiday column 

begin_date = "2023-01-01"
end_date = "2024-12-31" 
# ^ parameters to be set by yaml config later

date_df = pd.DataFrame({'Date':pd.date_range(start=begin_date, end=end_date)})

date_df['Day'] = date_df['Date'].dt.day
date_df['Month'] = date_df['Date'].dt.month
date_df['MonthName'] = date_df['Date'].dt.month_name()
date_df['Year'] = date_df['Date'].dt.year
date_df['DayOfWeek'] = date_df['Date'].dt.dayofweek
date_df['DayOfWeekName'] = date_df['Date'].dt.day_name()

In [106]:
holidays_df = pd.concat(map(pd.read_csv, ['raw_data/holidays/2023.csv', 'raw_data/holidays/2024.csv']))
# ^ change code above to run as for loop for each file inside the holidays folder

holidays_df = holidays_df.rename(columns={'Name': 'HolidayName'})

In [107]:
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])

In [108]:
date_merge_df = pd.merge(left=date_df, right=holidays_df, on=["Date"], how="left")

In [133]:
date_merge_df

,Date,Day,Month,MonthName,Year,DayOfWeek,DayOfWeekName,HolidayName
0,2023-01-01,1,1,January,2023,6,Sunday,NaN
1,2023-01-02,2,1,January,2023,0,Monday,New Year's Day
2,2023-01-03,3,1,January,2023,1,Tuesday,NaN
3,2023-01-04,4,1,January,2023,2,Wednesday,NaN
4,2023-01-05,5,1,January,2023,3,Thursday,NaN
...,...,...,...,...,...,...,...,...
726,2024-12-27,27,12,December,2024,4,Friday,NaN
727,2024-12-28,28,12,December,2024,5,Saturday,NaN
728,2024-12-29,29,12,December,2024,6,Sunday,NaN
729,2024-12-30,30,12,December,2024,0,Monday,NaN


In [157]:
ids = crime_df['case_']

In [159]:
crime_df[ids.isin(ids[ids.duplicated()])].sort_values("case_")

,:id,:created_at,:updated_at,:version,case_,date_of_occurrence,block,_iucr,_primary_decsription,_secondary_description,...,beat,ward,fbi_cd,x_coordinate,y_coordinate,latitude,longitude,location.latitude,location.longitude,location.human_address
2727,row-zmsy-ixvj-xnzj,2023-11-19T11:02:13.735Z,2023-11-19T11:02:20.179Z,rv-67qf~9cqy.qa7t,JG499426,2023-11-10T20:52:00.000,083XX S PULASKI RD,0110,HOMICIDE,FIRST DEGREE MURDER,...,834,18,01A,1151183,1848997,41.741527505,-87.721656705,41.741527505,-87.721656705,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2942,row-w2vg~rer6_4b8d,2023-11-19T11:02:13.735Z,2023-11-19T11:02:20.179Z,rv-eu85_6hs9_ag5t,JG499426,2023-11-10T17:16:00.000,083XX S PULASKI RD,0110,HOMICIDE,FIRST DEGREE MURDER,...,834,18,01A,1151183,1848997,41.741527505,-87.721656705,41.741527505,-87.721656705,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2975,row-t8a5-sknz.m7hf,2023-11-19T11:02:13.735Z,2023-11-19T11:02:20.179Z,rv-cp8i.qdy8.ccb2,JG499426,2023-11-10T17:22:00.000,083XX S PULASKI RD,0110,HOMICIDE,FIRST DEGREE MURDER,...,834,18,01A,1151183,1848997,41.741527505,-87.721656705,41.741527505,-87.721656705,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
